## Mô hình đề xuất anime dựa vào Cosine similarity

#### Mục tiêu: Dựa vào một bộ anime cho trước, ta sẽ tìm cách đề xuất cho người dùng một số bộ anime khác tương tự như bộ anime đó.

Ở đây ta sẽ chỉ sử dụng các thông tin nội tại của một bộ phim, nên ta chỉ xét các đặc trưng: <span style="color: orange;">'Genres', 'Type', 'Episodes', 'Producers', 'Studios', 'Source', 'Time per ep (Min)', 'Released date', 'Completed date', 'Rating'.</span>

---

## 1. Đọc dữ liệu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

Các PATH dùng trong bài này

In [2]:
ROOT = os.path.dirname(os.getcwd())
DATASET_FOLDER = os.path.join(ROOT, 'Data Preprocessing')
ANIME_CLEAN_DATASET = os.path.join(DATASET_FOLDER, 'anime-data-preprocessing.csv')

Đọc dữ liệu vào dataframe

In [3]:
df = pd.read_csv(ANIME_CLEAN_DATASET)
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               13834 non-null  object 
 1   Score              13834 non-null  float64
 2   Genres             13834 non-null  object 
 3   Synopsis           13834 non-null  object 
 4   Type               13834 non-null  object 
 5   Episodes           13834 non-null  float64
 6   Producers          13834 non-null  object 
 7   Studios            13834 non-null  object 
 8   Source             13834 non-null  object 
 9   Time per ep (Min)  13834 non-null  float64
 10  Rating             13834 non-null  object 
 11  Rank               13834 non-null  int64  
 12  Popularity         13834 non-null  int64  
 13  Favorites          13834 non-null  int64  
 14  Scored By          13834 non-null  int64  
 15  Members            13834 non-null  int64  
 16  Released date      138

,Name,Score,Genres,Synopsis,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Rank,Popularity,Favorites,Scored By,Members,Released date,Completed date
0,Fullmetal Alchemist: Brotherhood,9.10,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24.00,R - 17+ (violence & profanity),1,3,217606,2020030,3176556,2009-04-05,2010-07-04
1,Steins;Gate,9.07,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,24.0,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24.00,PG-13 - Teens 13 or older,2,13,182964,1336233,2440369,2011-04-06,2011-09-14
2,Bleach: Sennen Kessen-hen,9.07,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,13.0,"TV Tokyo, Aniplex, Dentsu, Shueisha",Pierrot,Manga,24.00,R - 17+ (violence & profanity),3,464,17999,213872,445198,2022-10-11,2022-12-27
3,Gintama°,9.06,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,51.0,"TV Tokyo, Aniplex, Dentsu",Bandai Namco Pictures,Manga,24.00,PG-13 - Teens 13 or older,4,331,15947,237957,595767,2015-04-08,2016-03-30
4,Shingeki no Kyojin Season 3 Part 2,9.05,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,10.0,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23.00,R - 17+ (violence & profanity),5,24,55245,1471825,2104016,2019-04-29,2019-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13829,Kokuhaku,2.30,Horror,"Beside a certain building, a girl appears to m...",ONA,1.0,UNKNOWN,UNKNOWN,Original,0.67,R - 17+ (violence & profanity),13830,7105,17,4904,6697,2015-08-27,2015-08-27
13830,Hametsu no Mars,2.22,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,1.0,"Idea Factory, King Records, Design Factory",WAO World,Visual novel,19.00,R - 17+ (violence & profanity),13831,2512,295,47630,65622,2005-07-06,2005-07-06
13831,Tsui no Sora,2.22,Hentai,"After the sudden death of a student, mysteriou...",OVA,1.0,Obtain Future,UNKNOWN,Visual novel,23.00,Rx - Hentai,13832,7563,24,3436,5713,2002-08-10,2002-08-10
13832,Utsu Musume Sayuri,1.98,"Avant Garde, Comedy",Sayuri is a curious creature who lives on her ...,OVA,1.0,UNKNOWN,UNKNOWN,Original,3.00,R+ - Mild Nudity,13833,4492,50,15873,20789,2003-01-01,2003-01-01


Tiến hành bỏ đi các đặc trưng <span style="color: red;">'Score', 'Synopsis', 'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members'</span>. 

In [4]:
columns_to_be_dropped = ['Score', 'Synopsis', 'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members']
df.drop(columns=columns_to_be_dropped, inplace=True)
df.head(1)

,Name,Genres,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Released date,Completed date
0,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy",TV,64.0,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24.0,R - 17+ (violence & profanity),2009-04-05,2010-07-04


Hai đặc trưng Released date và Completed date vẫn chưa được chuyển về dạng datetime nên ta sẽ chuyển ở đây luôn, các bước xử lý phức tạp hơn ta làm trong phần sau.

In [5]:
df['Released date'] = pd.to_datetime(df['Released date'], errors='coerce')
df['Completed date'] = pd.to_datetime(df['Completed date'], errors='coerce')

---

## 2. Tiền xử lý dữ liệu 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               13834 non-null  object        
 1   Genres             13834 non-null  object        
 2   Type               13834 non-null  object        
 3   Episodes           13834 non-null  float64       
 4   Producers          13834 non-null  object        
 5   Studios            13834 non-null  object        
 6   Source             13834 non-null  object        
 7   Time per ep (Min)  13834 non-null  float64       
 8   Rating             13834 non-null  object        
 9   Released date      13834 non-null  datetime64[ns]
 10  Completed date     13727 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(7)
memory usage: 1.2+ MB


Hiện tại bộ dữ liệu của chúng ta gồm 10 đặc trưng (không tính 'Name').

- Các đặc trưng dạng object gồm: <span style="color: orange;">'Genres', 'Type', 'Producers', 'Studios', 'Source', 'Rating'</span>.

- Các đặc trưng dạng số gồm: <span style="color: orange;">'Episodes', 'Time per ep (Min)'</span>.

- Các đặc trưng dạng datetime gồm: <span style="color: orange;">'Released date', 'Completed date'</span>.

Ta cần để ý các vấn đề quan trọng sau:
- Ba đặc trưng <span style="color: orange;">'Genres'</span>, <span style="color: orange;">'Studios'</span> và <span style="color: orange;"> 'Producers'</span> có dữ liệu kiểu: 'Action, Drama, Fantasy' hay 'TV Tokyo, Aniplex', tức là categorical đa nhãn. 
- Ba đặc trưng <span style="color: orange;">'Type'</span>, <span style="color: orange;">'Source'</span> và <span style="color: orange;">'Rating'</span> là kiểu categorical đơn nhãn.
- Đặc trưng <span style="color: orange;">'Completed date'</span> có lẫn một số giá trị NaT, tức bộ anime vẫn đang chiếu.
- Ba đặc trưng dạng object <span style="color: orange;">'Producers'</span>, <span style="color: orange;">'Studios'</span> và <span style="color: orange;">'Source'</span> vẫn còn lẫn một số giá trị 'UNKNOWN'.
- Hai đặc trưng dạng số: <span style="color: orange;">'Episodes'</span> và <span style="color: orange;">'Time per ep (Min)'</span> cần được chuẩn hóa.

### 2.1 Xử lý các đặc trưng còn lẫn 'UNKNOWN'

In [7]:
# Kiểm tra số lượng giá trị 'UNKNOWN' có trong một cột
def count_unknown_values(column):
    return column.str.upper().str.contains('UNKNOWN').sum()

# Lưu ý ở đây ta biết rõ chỉ còn 2 feature 'Producers', 'Studios', 'Source' là có giá trị 'UNKNOWN'
unknown_count = df[['Producers', 'Studios', 'Source']].apply(count_unknown_values)
unknown_count

Producers    5018
Studios      2345
Source       1508
dtype: int64

- Do feature <span style="color: orange;"> 'Producers'</span> có quá nhiều giá trị thiếu ('UNKNOWN'), ta xem xét bỏ luôn feature đó ra khỏi dataset.

- Với hai feature còn lại ta sẽ drop các sample chứa giá trị thiếu

In [8]:
# Drop the 'Producers' column
df.drop(columns=['Producers'], inplace=True)

# Drop các sample có giá trị 'UNKNOWN' trong 'Studios' và 'Source'
mask = df['Studios'].str.upper().str.contains('UNKNOWN') | df['Source'].str.upper().str.contains('UNKNOWN')
df = df[~mask]
df = df.reset_index(drop=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               10562 non-null  object        
 1   Genres             10562 non-null  object        
 2   Type               10562 non-null  object        
 3   Episodes           10562 non-null  float64       
 4   Studios            10562 non-null  object        
 5   Source             10562 non-null  object        
 6   Time per ep (Min)  10562 non-null  float64       
 7   Rating             10562 non-null  object        
 8   Released date      10562 non-null  datetime64[ns]
 9   Completed date     10468 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(6)
memory usage: 825.3+ KB


### 2.2 Xử lý các đặc trưng categorical đơn nhãn

Đối với 2 đặc trưng là <span style="color: orange;">'Type'</span> và <span style="color: orange;">'Source'</span>, do chúng không có Tính thứ tự nên ta dùng <span style="color: orange;">One-hot encoding</span> (Ở đây dùng sẵn hàm <span style="color: orange;">Dummy encoding</span> của pandas cho tiện)

In [10]:
type_dummies = df['Type'].str.get_dummies()
source_dummies = df['Source'].str.get_dummies()

# Thêm dữ liệu mới tạo vào df cũ
df = pd.concat([df, type_dummies, source_dummies], axis=1)

# Drop các cột 'Type', 'Source' sau khi đã tạo dummies
df.drop(columns=['Type', 'Source'], inplace=True)

Đối với đặc trưng <span style="color: orange;">'Rating'</span>, do chúng có Tính thứ tự nên ta sẽ dùng <span style="color: orange;">Label encoding</span>

In [11]:
# Tạo encoder
encoder = LabelEncoder()
df['Rating'] = encoder.fit_transform(df['Rating'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Name               10562 non-null  object        
 1   Genres             10562 non-null  object        
 2   Episodes           10562 non-null  float64       
 3   Studios            10562 non-null  object        
 4   Time per ep (Min)  10562 non-null  float64       
 5   Rating             10562 non-null  int32         
 6   Released date      10562 non-null  datetime64[ns]
 7   Completed date     10468 non-null  datetime64[ns]
 8   Movie              10562 non-null  int64         
 9   Music              10562 non-null  int64         
 10  ONA                10562 non-null  int64         
 11  OVA                10562 non-null  int64         
 12  Special            10562 non-null  int64         
 13  TV                 10562 non-null  int64         
 14  4-koma

### 2.3 Xử lý các đặc trưng categorical đa nhãn

Ta chuyển dữ liệu thành một vector chứa các giá trị nhị phân

Ví dụ nếu bài toán có tổng cộng 3 class: Dog, Cat, Bird. Một dữ liệu có giá trị 'Dog, Cat' sẽ được biểu diễn bằng vector [1, 1, 0]

In [13]:
# Trước tiên ta cần tách các giá trị string thành một list các class
df['Genres'] = df['Genres'].apply(lambda x: [x.strip() for x in x.split(',')])
df['Studios'] = df['Studios'].apply(lambda x: [x.strip() for x in x.split(',')])

In [14]:
# Sử dụng mulitlabel binarizer của sklearn
mlbinarizer = MultiLabelBinarizer()


# Feature Studios
# fit_transform cột Studios thành một ma trận nhị phân
altered_Studios_matrix = mlbinarizer.fit_transform(df['Studios'])
# Chuyển ma trận đó thành DataFrame
altered_Studios_df = pd.DataFrame(altered_Studios_matrix, columns=mlbinarizer.classes_)
# Gắn dataframe mới vào dataframe cũ
df = pd.concat([df, altered_Studios_df], axis=1)


mlbinarizer = MultiLabelBinarizer()
# Feature Genres
# fit_transform cột Genres thành một ma trận nhị phân
altered_Genres_matrix = mlbinarizer.fit_transform(df['Genres'])
# Chuyển ma trận đó thành DataFrame
altered_Genres_df = pd.DataFrame(altered_Genres_matrix, columns=mlbinarizer.classes_)
# Gắn dataframe mới vào dataframe cũ
df = pd.concat([df, altered_Genres_df], axis=1)

In [15]:
# Xóa các cột 'Genres', 'Studios' 
df.drop(columns=['Genres', 'Studios'], inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Columns: 899 entries, Name to Suspense
dtypes: datetime64[ns](2), float64(2), int32(872), int64(22), object(1)
memory usage: 37.3+ MB


### 2.4 Xử lý các đặc trưng datetime

Để cho đơn giản, đối với hai đặc trưng <span style="color: orange;">'Released date'</span> và <span style="color: orange;">'Completed date'</span>, ta sẽ chỉ giữ lại phần <span style="color: orange;">năm</span>. 

Với 'Completed date' nếu gặp dữ liệu dạng NaT (bộ phim này vẫn đang chiếu), ta sẽ gán năm là 2023, tức năm mà bộ dataset này được tạo ra.

In [17]:
df['Released date'] = df['Released date'].dt.year

df['Completed date'] = df['Completed date'].dt.year
df['Completed date'] = df['Completed date'].fillna(2023)

# Chuyển về kiểu int cho đồng bộ
df['Released date'] = df['Released date'].astype(int)
df['Completed date'] = df['Completed date'].astype(int)

### 2.5 Chuẩn hóa các đặc trưng dạng số

In [18]:
# Các cột cần chuẩn hóa
numeric_features = ['Episodes', 'Time per ep (Min)', 'Released date', 'Completed date']

# Tạo scaler
scaler = StandardScaler()

# Chuẩn hóa các cột số
df[numeric_features] = scaler.fit_transform(df[numeric_features])

---

## 3. Tạo mô hình

Tạo ma trận cosine similarity giữa tất cả các phim với nhau

In [19]:
anime_info_df = df.drop(columns='Name')
cosine_similarity_matrix = cosine_similarity(anime_info_df, anime_info_df)

Tạo hàm đề xuất phim, ở đây ta sẽ lấy input là tên phim và so sánh cosine similarity giữa thông tin của phim đó với tất cả các phim khác, sau cùng chọn ra các phim có độ tương đồng cao nhất

In [20]:
def recommend_anime(anime_name, top_n=10):

    # Lấy index của bộ anime đầu vào
    input_anime_sample = df[df['Name'] == anime_name]

    if input_anime_sample.empty:
        print('Không có thông tin về bộ anime này.')
        return
    
    input_anime_index = input_anime_sample.index[0]
    

    # Tính cosine similarity giữa bộ anime đầu vào và tất cả các bộ anime khác
    similarities = list(enumerate(cosine_similarity_matrix[input_anime_index]))

    # Sort các giá trị similarity
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Lấy top N bộ anime 
    top_n_anime = similarities[1:top_n+1]

    # Lấy tên của top N bộ anime
    top_n_anime_index = [anime[0] for anime in top_n_anime]
    top_n_anime_names = df.iloc[top_n_anime_index]['Name']

    print(f'Top {top_n} bộ anime bạn nên xem nếu bạn thích {anime_name}:')
    for i, anime_name in enumerate(top_n_anime_names):
        print(f'{i+1}. {anime_name}')

Test thử

In [21]:
recommend_anime('Naruto')

Top 10 bộ anime bạn nên xem nếu bạn thích Naruto:
1. Bleach
2. Yu☆Gi☆Oh! Duel Monsters
3. Boruto: Naruto Next Generations
4. Dragon Ball Z
5. Naruto: Shippuuden
6. InuYasha
7. Fairy Tail
8. Yu☆Gi☆Oh! Duel Monsters GX
9. Katekyo Hitman Reborn!
10. Hunter x Hunter (2011)


In [22]:
recommend_anime('Chainsaw Man', 7)

Top 7 bộ anime bạn nên xem nếu bạn thích Chainsaw Man:
1. Jigokuraku
2. Heion Sedai no Idaten-tachi
3. Jujutsu Kaisen
4. Katsute Kami Datta Kemono-tachi e
5. Dorohedoro
6. Shingeki no Kyojin: The Final Season Part 2
7. Kimetsu no Yaiba: Katanakaji no Sato-hen


In [23]:
recommend_anime('Kimi no Na wa.', 6)

Top 6 bộ anime bạn nên xem nếu bạn thích Kimi no Na wa.:
1. Tenki no Ko
2. Giovanni no Shima
3. Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. Movie
4. Nakitai Watashi wa Neko wo Kaburu
5. Sora no Aosa wo Shiru Hito yo
6. Kimi to, Nami ni Noretara


In [24]:
recommend_anime('Overlord')

Top 10 bộ anime bạn nên xem nếu bạn thích Overlord:
1. Nejimaki Seirei Senki: Tenkyou no Alderamin
2. Overlord III
3. Overlord II
4. Overlord IV
5. Mahou Sensou
6. Gate: Jieitai Kanochi nite, Kaku Tatakaeri
7. Gate: Jieitai Kanochi nite, Kaku Tatakaeri Part 2
8. Kamisama no Inai Nichiyoubi
9. Goblin Slayer
10. Sword Art Online: Alicization - War of Underworld


In [25]:
recommend_anime('Howl no Ugoku Shiro')

Top 10 bộ anime bạn nên xem nếu bạn thích Howl no Ugoku Shiro:
1. Karigurashi no Arrietty
2. Gake no Ue no Ponyo
3. Majo no Takkyuubin
4. Toki wo Kakeru Shoujo
5. Neko no Ongaeshi
6. Tibet Inu Monogatari
7. Brave Story
8. Mimi wo Sumaseba
9. Kappa no Coo to Natsuyasumi
10. Guskou Budori no Denki (2012)
